# **Section 1 - B**

----------

## Sample, auto-detect the device

At the end of the previous section, we had loaded the pretrained gpt2 model and its weights into our architecture and generated the model. But now we could like to initialize our own weights, we want the model to weights to be generated randomly.

So that can be done fairly simple way:

```
#model = GPT.from_pretrained("gpt2")
model = GPT(GPTConfig())
```

we just call our default `GPTConfig()` that we made. So what PyTorch does is that, it internally assigns random weights to each of the layers in our config, therefore we can use this to generate text from our model.

Lastly, before i run this, we also added an additional line of code to better control the device used to run this model. In my case i do have a GPU with CUDA capability. So, if you want to run the model until this point you can also do that using CPU. We have added this additional flag point just to show which device you are using here:

```
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

print(f"Device used: {device})
```

The rest of the code follows a dynamic approach of detecting the device as well (even in the `forward()` you will see that we have used `device=idx.device`), therefore we are ensuring that all the layers are using the same device while generating.

And this is the final output that we generated!

![Sampling and Auto detect output](assets/auto-device-output.png)

Obviously it is gibberish lol, we will get to the training next!

&nbsp;

## Let’s train: data batches (B,T) → logits (B,T,C)

> **NOTE**
>
>We will be loading our dataset now. Sensei used his fav "The tiny Shakespear dataset", I am going ahead and using MY Favourite dataset which is what i also used for my GPT-1 implementation which is the HARRY POTTER NOVELS COLLECTION dataset. I directly took the `cleaned_dataset.txt` file which i had processed.
>
>If you want to see a simple breakdown version of the dataset and what we are about to do, take a look at [this notebook](https://github.com/MuzzammilShah/GPT-TransformerModel-2/blob/main/section-1b-dataset.ipynb) on my repo.